## Обучение опций 
Нашей задачей будет создание набора опций, каждая из которых должна быть обучена достигать определенные состояния в задаче такси.Обученные опции затем могут быть применены для создания и обучения иерархии. Мы будем использовать QLearningAgent, которого мы написали на одном из прошлых семинаров. 

In [1]:
# импортируем файлы и создаем окружение
import gym
import random
from collections import defaultdict

import numpy as np
environment = gym.make('Taxi-v2')
environment.render()




+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
# создаем классс для Q-агента
class QLearningAgent:
    def __init__(self, alpha, epsilon, gamma, get_legal_actions):
        self.get_legal_actions = get_legal_actions
        self._q_values = defaultdict(lambda: defaultdict(lambda: 0))  # when called, non-existent values appear as zeros
        self.alpha = alpha
        self.epsilon = epsilon
        self.gamma = gamma

    def get_q_value(self, state, action):
        """
          Returns Q(state,action)
        """
        return self._q_values[state][action]

    def set_q_value(self, state, action, value):
        """
          Sets the Qvalue for [state,action] to the given value
        """
        self._q_values[state][action] = value

    def get_value(self, state):
        """
          Returns max_action Q(state,action)
          where the max is over legal actions.
        """

        possible_actions = self.get_legal_actions(state)
        # If there are no legal actions, return 0.0
        if len(possible_actions) == 0:
            return 0.0

        value = max([self.get_q_value(state, action) for action in possible_actions])
        return value

    def get_policy(self, state):
        """
          Compute the best action to take in a state.

        """
        possible_actions = self.get_legal_actions(state)

        # If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None

        best_action = None

        for action in possible_actions:
            if best_action is None:
                best_action = action
            elif self.get_q_value(state, action) > self.get_q_value(state, best_action):
                best_action = action

        return best_action

    def get_action(self, state):
        """
          Compute the action to take in the current state, including exploration.

          With probability self.epsilon, we should take a random action.
          otherwise - the best policy action (self.getPolicy).

        """

        #
        possible_actions = self.get_legal_actions(state)

        # если в текущей ситуации нет возможных действий - возвращаем None
        if len(possible_actions) == 0:
            return None

        if np.random.random() < self.epsilon:
            action = random.choice(possible_actions)
        else:
            action = self.get_policy(state)
        return action

    def update(self, state, action, next_state, reward):
        t = self.alpha * (reward + self.gamma * self.get_value(next_state) - self.get_q_value(state, action))
        reference_qvalue = self.get_q_value(state, action) + t
        self.set_q_value(state, action, reference_qvalue)

### Задание 1 
Разберемся как реализована среда Taxi: https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py

Создадим 4 окружения аналогичных Taxi, в которых целью агента будет достижение одной из точек: R, G, B, Y соответственно. 

In [4]:
class TaxiStepWrapper(gym.Wrapper):

    def __init__(self, env, target_id, target_reward):
        """
        target_id - индекс целевой точки в окружении такси (см self.locs = locs = [(0,0), (0,4), (4,0), (4,3)])
        target_reward - вознаграждение за достижение цели
        """
        super().__init__(env)
        self._target = target_id
        self._target_reward = target_reward

    def _step(self, action):
        # получаем изначальные параметры (state, reward, _, obs), которые передает среда, используя метод step 
        # проверяем является ли полученнуе состояние завершающим для нашего модифицированного окружения
        # изменяем вознаграждение (reward) и флаг завершения эпизода (is_done)
        # за каждое действие будем давать вознаграждение -1, за достижение цели - self._target_reward
        # Ваш код здесь
        
        return state, reward, is_done, obs

Проверим наши среды, используя случайную стратегию.  Порядок точек должен быть  R, G, Y, B.

In [254]:
for target in range(4):
    # создаем окружение с заданным целевым состоянием
    # Ваш код здесь
    
    # применяем случаную стратегию, пока эпизод не завершится
    # Ваш код здесь
    
    wrapped_env.render()
    print("state:{s} reward:{r}\n".format(**locals()))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
state:18 reward:50

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
state:87 reward:50

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
state:416 reward:50

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
state:469 reward:50



In [5]:
# воспользуеся реализованной на предыдущем семинаре функцией
def play_and_train(env, agent, t_max=10 ** 4):
    total_discounted_reward = 0.0
    s = env.reset()
    for t in range(t_max):
        a = agent.get_action(s)
        next_s, r, done, _ = env.step(a)
        agent.update(s, a, next_s, r)
        s = next_s
        total_discounted_reward += r
        if done:
            break
    return total_discounted_reward

### Задание 2 
1. Обучим агентов на созданных нами окружениях.
2. Создадим упрощенный вариант опций, каждая опция будет иметь стратегию, множество начальных и конечных состояний.

In [256]:
n_actions = environment.action_space.n

# параметры, которые будут использовать агенты
params = {"alpha": 0.1, "epsilon": 0.1, "gamma": 0.99, "get_legal_actions": lambda s: range(4)}

# создаем агентов 
agents_for_options = [QLearningAgent(**params) for _ in range(4)]


for index in range(4):
    # используя созданных окружения обучаем на них агентов
    # Ваш код здесь

In [257]:
# реализуем класс опции
class Option:
    def __init__(self, policy, termination, initial):
        """
            policy - стратегия, у которой должны быть методы update и get_action
            initial- множество состояний, в которых опция может быть запущена
            termination - множество состояний, в которых опция должна быть завершена (упрощенная версия)
        """
        self.policy = policy
        self.termination_lambda = termination
        self.initial_lambda = initial

In [258]:
options = []
for index, agent in enumerate(agents_for_options):
    # Испоьзуя обученных агентов создаем опции. 
    # Начальными состоянми зададим все кроме целевых
    # Ваш код здесь 
    
    options.append(Option(policy=agent, termination=, initial=))


### Задание 3
Напишем функцию, которая будет запускать опцию и возвращать дисконтированное вознаграждения, опираясь на число совершенных действий. 
$$ R = r_{1} + \gamma r_{2} + \gamma^{2} r_{3} + \dots + \gamma^{t-1}r_{t}$$

In [259]:
def go_option(s, option, gamma, env):
    """
    option - опция, стратегию которой будем использовать
    gamma - дисконтирующий коэффициент
    env - среда
    """
    reward = 0
    steps = 0
    
    # Ваш код здесь
    # используем метод get_action, стратегии нашей опции, взаимодействуем с окружением и сохраняем правильное вознаграждение

    return reward

In [260]:
env = gym.make('Taxi-v2')
s = env.reset()

env.render()
r = go_option(s, options[0], 0.99, env)
env.render()
print("smdp reward: {r}".format(**locals()))


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
smdp reward: -1.0


Кажется, что все хорошо, но мы забыли рассмотреть вариант, когда пассажир может находиться в такси! Переведем среду в состояние, где пассажира мы уже подобрали и посмотрим как ведет одна из опций.

In [261]:
s = env.reset()
while list(env.unwrapped.decode(s))[2] != 4:
    s, _, _, _ = env.step(random.randint(0, 5))
    
env.render()
r = go_option(s, options[0], 0.99, env)
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


### Задание 4
Видим, что опции не обучились действовать в такой ситуации. 
Исправим нашу функцию обучения так, чтобы опции работали корректно для всех возможных состояний среды и сгенерируем их заново.

In [262]:
agents_for_options = [QLearningAgent(**params) for _ in range(4)]
for index in range(4):
    pass
    # Ваш код здесь
    
# Ваш код здесь

Проверим работу:

In [263]:
s = env.reset()
while list(env.unwrapped.decode(s))[2] != 4:
    s, _, _, _ = env.step(random.randint(0, 5))
    
env.render()
r = go_option(s, options[0], 0.99, env)
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (South)


### Бонус 
Реализуйте иерархию, используя элементарные (опции из одного действия) и обученные опции.